In [ ]:
# Import
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from finta import TA
from talib import RSI, BBANDS

# Allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# import holoviews as hv
# hv.extension('bokeh')

# # selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [2]:
# Imports required for automated plot to PNG saves (one time use)

# import holoviews as hv
# hv.extension('bokeh')

# # selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [3]:
# Read in the CSV files that contain SMA, EMA, and BB/RSI inidcators based on ratio data
googl_df = pd.read_csv("Resources/googl_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [4]:
# Create a list of tuples containing the stock DataFrames and their respective columns for iteration
ticker_data_all = [
    (googl_df, [("GOOGL P/S (LTM)", "ps"), ("GOOGL P/FCF (LTM)", "pfcf"), ("GOOGL P/E (LTM)", "pe")]),
    (nvda_df, [("NVDA P/S (LTM)", "ps"), ("NVDA P/FCF (LTM)", "pfcf"), ("NVDA P/E (LTM)", "pe")]),
    (mmm_df, [("MMM P/S (LTM)", "ps"), ("MMM P/FCF (LTM)", "pfcf"), ("MMM P/E (LTM)", "pe")]),
    (pg_df, [("PG P/S (LTM)", "ps"), ("PG P/FCF (LTM)", "pfcf"), ("PG P/E (LTM)", "pe")]),
]

In [5]:
# Create and Populate the "signal" column for signal DataFrames
for df, metrics in ticker_data_all:
    for column_name, prefix, in metrics:
        df[f'{prefix}_Signal_sma'] = 0.0
        df[f'{prefix}_Signal_ema'] = 0.0
        df[f'{prefix}_Signal_sma30'] = 0.0
        df[f'{prefix}_Signal_ema30'] = 0.0

In [6]:
# Generate the trading signal 0 or 1,
# where 1 is when sma_fast is greater than sma_slow or ema_fast is greater than ema_slow
# and 0 otherwise
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        # Apply the condition to set the 'Signal' column
        df[f'{prefix}_Signal_sma'] = np.where(df[f'{prefix}_sma_slow'] > df[f'{prefix}_sma_fast'], 1.0, 0.0)
        df[f'{prefix}_Signal_ema'] = np.where(df[f'{prefix}_ema_slow'] > df[f'{prefix}_ema_fast'], 1.0, 0.0)
        df[f'{prefix}_Signal_sma30'] = np.where(df[f'{prefix}_sma_slow'] > df[f'{prefix}_sma_fast30'], 1.0, 0.0)
        df[f'{prefix}_Signal_ema30'] = np.where(df[f'{prefix}_ema_slow'] > df[f'{prefix}_ema_fast30'], 1.0, 0.0)

# Note: SMA and EMA fast and slow signals positions have been reversed in the signal equasion to reflect the fact that
# Note: they have been calculated based on the ratio data (P/S, P/FCF, P/E) where a lower value is preferred

In [7]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Entry/Exit_sma'] = df[f'{prefix}_Signal_sma'].diff()
        df[f'{prefix}_Entry/Exit_ema'] = df[f'{prefix}_Signal_ema'].diff()
        df[f'{prefix}_Entry/Exit_sma30'] = df[f'{prefix}_Signal_sma30'].diff()
        df[f'{prefix}_Entry/Exit_ema30'] = df[f'{prefix}_Signal_ema30'].diff()

### Investment Capital Tracking

In [8]:
# Set initial investment capital
initial_capital = float(100000)

# Set the share size per transaction
share_size = 500

In [9]:
# Creating a position column to store the number of shares held
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Position_sma'] = share_size * df[f'{prefix}_Signal_sma']
        df[f'{prefix}_Position_ema'] = share_size * df[f'{prefix}_Signal_ema']
        df[f'{prefix}_Position_sma30'] = share_size * df[f'{prefix}_Signal_sma30']
        df[f'{prefix}_Position_ema30'] = share_size * df[f'{prefix}_Signal_ema30']

In [10]:
# Determine the points in time where the share position is bought or sold
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Entry/Exit Position_sma'] = df[f'{prefix}_Position_sma'].diff()
        df[f'{prefix}_Entry/Exit Position_ema'] = df[f'{prefix}_Position_ema'].diff()
        df[f'{prefix}_Entry/Exit Position_sma30'] = df[f'{prefix}_Position_sma30'].diff()
        df[f'{prefix}_Entry/Exit Position_ema30'] = df[f'{prefix}_Position_ema30'].diff()

In [11]:
# Multiply the close price by the number of shares held, or the Position
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        adj_close_col = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        df[f'{prefix}_Portfolio Holdings_sma'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_sma']
        df[f'{prefix}_Portfolio Holdings_ema'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_ema']
        df[f'{prefix}_Portfolio Holdings_sma30'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_sma30']
        df[f'{prefix}_Portfolio Holdings_ema30'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_ema30']

In [12]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        adj_close_col = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        df[f'{prefix}_Portfolio Cash_sma'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_sma']).cumsum()
        df[f'{prefix}_Portfolio Cash_ema'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_ema']).cumsum()
        df[f'{prefix}_Portfolio Cash_sma30'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_sma30']).cumsum()
        df[f'{prefix}_Portfolio Cash_ema30'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_ema30']).cumsum()

In [13]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Ticker Total_sma'] = df[f'{prefix}_Portfolio Cash_sma'] + df[f'{prefix}_Portfolio Holdings_sma']
        df[f'{prefix}_Ticker Total_ema'] = df[f'{prefix}_Portfolio Cash_ema'] + df[f'{prefix}_Portfolio Holdings_ema']
        df[f'{prefix}_Ticker Total_sma30'] = df[f'{prefix}_Portfolio Cash_sma30'] + df[f'{prefix}_Portfolio Holdings_sma30']
        df[f'{prefix}_Ticker Total_ema30'] = df[f'{prefix}_Portfolio Cash_ema30'] + df[f'{prefix}_Portfolio Holdings_ema30']

In [14]:
# Calculate the portfolio daily returns
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Daily Returns_sma'] = df[f'{prefix}_Ticker Total_sma'].pct_change()
        df[f'{prefix}_Daily Returns_ema'] = df[f'{prefix}_Ticker Total_ema'].pct_change()
        df[f'{prefix}_Daily Returns_sma30'] = df[f'{prefix}_Ticker Total_sma30'].pct_change()
        df[f'{prefix}_Daily Returns_ema30'] = df[f'{prefix}_Ticker Total_ema30'].pct_change()

In [15]:
# Calculate the portfolio cumulative returns
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Cumulative Returns_sma'] = (1 + df[f'{prefix}_Daily Returns_sma']).cumprod() - 1
        df[f'{prefix}_Cumulative Returns_ema'] = (1 + df[f'{prefix}_Daily Returns_ema']).cumprod() - 1
        df[f'{prefix}_Cumulative Returns_sma30'] = (1 + df[f'{prefix}_Daily Returns_sma30']).cumprod() - 1
        df[f'{prefix}_Cumulative Returns_ema30'] = (1 + df[f'{prefix}_Daily Returns_ema30']).cumprod() - 1

In [16]:
# Concatenate the total value per stock position into a single DataFrame

#SMA 10/100s
sma10_ps_ticker_totals = pd.concat([googl_df['ps_Ticker Total_sma'], nvda_df['ps_Ticker Total_sma'], 
                                   mmm_df['ps_Ticker Total_sma'], pg_df['ps_Ticker Total_sma']], axis="columns", join="inner")
sma10_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma10_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Ticker Total_sma'], nvda_df['pfcf_Ticker Total_sma'], 
                                   mmm_df['pfcf_Ticker Total_sma'], pg_df['pfcf_Ticker Total_sma']], axis="columns", join="inner")
sma10_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma10_pe_ticker_totals = pd.concat([googl_df['pe_Ticker Total_sma'], nvda_df['pe_Ticker Total_sma'], 
                                   mmm_df['pe_Ticker Total_sma'], pg_df['pe_Ticker Total_sma']], axis="columns", join="inner")
sma10_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

#SMA 30/100s
sma30_ps_ticker_totals = pd.concat([googl_df['ps_Ticker Total_sma30'], nvda_df['ps_Ticker Total_sma30'],
                                   mmm_df['ps_Ticker Total_sma30'], pg_df['ps_Ticker Total_sma30']], axis="columns", join="inner")
sma30_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma30_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Ticker Total_sma30'], nvda_df['pfcf_Ticker Total_sma30'],
                                   mmm_df['pfcf_Ticker Total_sma30'], pg_df['pfcf_Ticker Total_sma30']], axis="columns", join="inner")
sma30_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma30_pe_ticker_totals = pd.concat([googl_df['pe_Ticker Total_sma30'], nvda_df['pe_Ticker Total_sma30'],
                                   mmm_df['pe_Ticker Total_sma30'], pg_df['pe_Ticker Total_sma30']], axis="columns", join="inner")
sma30_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

# EMA 10/100s
ema10_ps_ticker_totals = pd.concat([googl_df['ps_Ticker Total_ema'], nvda_df['ps_Ticker Total_ema'],
                                   mmm_df['ps_Ticker Total_ema'], pg_df['ps_Ticker Total_ema']], axis="columns", join="inner")
ema10_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema10_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Ticker Total_ema'], nvda_df['pfcf_Ticker Total_ema'],
                                   mmm_df['pfcf_Ticker Total_ema'], pg_df['pfcf_Ticker Total_ema']], axis="columns", join="inner")
ema10_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema10_pe_ticker_totals = pd.concat([googl_df['pe_Ticker Total_ema'], nvda_df['pe_Ticker Total_ema'],
                                   mmm_df['pe_Ticker Total_ema'], pg_df['pe_Ticker Total_ema']], axis="columns", join="inner")
ema10_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

# EMA 30/100s
ema30_ps_ticker_totals = pd.concat([googl_df['ps_Ticker Total_ema30'], nvda_df['ps_Ticker Total_ema30'],
                                   mmm_df['ps_Ticker Total_ema30'], pg_df['ps_Ticker Total_ema30']], axis="columns", join="inner")
ema30_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema30_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Ticker Total_ema30'], nvda_df['pfcf_Ticker Total_ema30'],
                                   mmm_df['pfcf_Ticker Total_ema30'], pg_df['pfcf_Ticker Total_ema30']], axis="columns", join="inner")
ema30_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema30_pe_ticker_totals = pd.concat([googl_df['pe_Ticker Total_ema30'], nvda_df['pe_Ticker Total_ema30'],
                                   mmm_df['pe_Ticker Total_ema30'], pg_df['pe_Ticker Total_ema30']], axis="columns", join="inner")
ema30_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']


In [17]:
# Calculate and plot the sum of the ticker total values for each stock
# sma10_portfolio_total['portfolio_total'] = sma10_ticker_totals.sum(axis=1)
# sma30_portfolio_total['portfolio_total'] = sma30_ticker_totals.sum(axis=1)
# ema10_portfolio_total['portfolio_total'] = ema10_ticker_totals.sum(axis=1)
# ema30_portfolio_total['portfolio_total'] = ema30_ticker_totals.sum(axis=1)

sma10_ps_ticker_totals['portfolio_total'] = sma10_ps_ticker_totals.sum(axis=1)
sma10_pfcf_ticker_totals['portfolio_total'] = sma10_pfcf_ticker_totals.sum(axis=1)
sma10_pe_ticker_totals['portfolio_total'] = sma10_pe_ticker_totals.sum(axis=1)

sma30_ps_ticker_totals['portfolio_total'] = sma30_ps_ticker_totals.sum(axis=1)
sma30_pfcf_ticker_totals['portfolio_total'] = sma30_pfcf_ticker_totals.sum(axis=1)
sma30_pe_ticker_totals['portfolio_total'] = sma30_pe_ticker_totals.sum(axis=1)

ema10_ps_ticker_totals['portfolio_total'] = ema10_ps_ticker_totals.sum(axis=1)
ema10_pfcf_ticker_totals['portfolio_total'] = ema10_pfcf_ticker_totals.sum(axis=1)
ema10_pe_ticker_totals['portfolio_total'] = ema10_pe_ticker_totals.sum(axis=1)

ema30_ps_ticker_totals['portfolio_total'] = ema30_ps_ticker_totals.sum(axis=1)
ema30_pfcf_ticker_totals['portfolio_total'] = ema30_pfcf_ticker_totals.sum(axis=1)
ema30_pe_ticker_totals['portfolio_total'] = ema30_pe_ticker_totals.sum(axis=1)

In [18]:
# Plot the total portfolio value for each stock per strategy per ratio

# SMA 10/100
plot_sma10_ps_ratios = sma10_ps_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma10_pfcf_ratios = sma10_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma10_pe_ratios = sma10_pe_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

# SMA 30/100
plot_sma30_ps_ratios = sma30_ps_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma30_pfcf_ratios = sma30_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma30_pe_ratios = sma30_pe_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

# EMA 10/100
plot_ema10_ps_ratios = ema10_ps_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema10_pfcf_ratios = ema10_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema10_pe_ratios = ema10_pe_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

# EMA 30/100
plot_ema30_ps_ratios = ema30_ps_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema30_pfcf_ratios = ema30_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema30_pe_ratios = ema30_pe_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

In [19]:
# Plot the total portfolio value for all stocks combined per stratgegy per ratio

# SMA 10/100
plot_sma10_ps_price_portfolio = sma10_ps_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma10_pfcf_price_portfolio = sma10_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 10/100 Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma10_pe_price_portfolio = sma10_pe_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

# SMA 30/100
plot_sma30_ps_price_portfolio = sma30_ps_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma30_pfcf_price_portfolio = sma30_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma30_pe_price_portfolio = sma30_pe_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

# EMA 10/100
plot_ema10_ps_price_portfolio = ema10_ps_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema10_pfcf_price_portfolio = ema10_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema10_pe_price_portfolio = ema10_pe_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

# EMA 30/100
plot_ema30_ps_price_portfolio = ema30_ps_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema30_pfcf_price_portfolio = ema30_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema30_pe_price_portfolio = ema30_pe_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

In [20]:
# Display the SMA and EMA strategy total portfolio value plots
display(plot_sma10_ps_ratios)
display(plot_sma10_pfcf_ratios)
display(plot_sma10_pe_ratios)

display(plot_sma30_ps_ratios)
display(plot_sma30_pfcf_ratios)
display(plot_sma30_pe_ratios)

display(plot_ema10_ps_ratios)
display(plot_ema10_pfcf_ratios)
display(plot_ema10_pe_ratios)

display(plot_ema30_ps_ratios)
display(plot_ema30_pfcf_ratios)
display(plot_ema30_pe_ratios)

display(plot_sma10_ps_price_portfolio)
display(plot_sma10_pfcf_price_portfolio)
display(plot_sma10_pe_price_portfolio)

display(plot_sma30_ps_price_portfolio)
display(plot_sma30_pfcf_price_portfolio)
display(plot_sma30_pe_price_portfolio)

display(plot_ema10_ps_price_portfolio)
display(plot_ema10_pfcf_price_portfolio)
display(plot_ema10_pe_price_portfolio)

display(plot_ema30_ps_price_portfolio)
display(plot_ema30_pfcf_price_portfolio)
display(plot_ema30_pe_price_portfolio)


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

...

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

In [21]:
import pandas as pd

# Concatenate 'portfolio_total' columns from each DataFrame into a new DataFrame
all_portfolio_totals = pd.concat([
    sma10_ps_ticker_totals['portfolio_total'].rename('sma10_ps'),
    sma10_pfcf_ticker_totals['portfolio_total'].rename('sma10_pfcf'),
    sma10_pe_ticker_totals['portfolio_total'].rename('sma10_pe'),
    sma30_ps_ticker_totals['portfolio_total'].rename('sma30_ps'),
    sma30_pfcf_ticker_totals['portfolio_total'].rename('sma30_pfcf'),
    sma30_pe_ticker_totals['portfolio_total'].rename('sma30_pe'),
    ema10_ps_ticker_totals['portfolio_total'].rename('ema10_ps'),
    ema10_pfcf_ticker_totals['portfolio_total'].rename('ema10_pfcf'),
    ema10_pe_ticker_totals['portfolio_total'].rename('ema10_pe'),
    ema30_ps_ticker_totals['portfolio_total'].rename('ema30_ps'),
    ema30_pfcf_ticker_totals['portfolio_total'].rename('ema30_pfcf'),
    ema30_pe_ticker_totals['portfolio_total'].rename('ema30_pe')
], axis=1)


In [22]:
all_portfolio_totals_final_day = all_portfolio_totals.tail(1)

In [23]:
# Transpose the DataFrame to turn columns into rows
transposed_df = all_portfolio_totals_final_day.T

# Sort the transposed DataFrame by its values in ascending order
sorted_df = transposed_df.sort_values(by=transposed_df.columns[0], ascending=True)

# Transposing back is optional and depends on how you intend to use or plot the data next
# sorted_df = sorted_df.T


In [24]:
# Assuming hvplot has been imported and is available
plot = sorted_df.hvplot.bar(
    title="Final Portfolio Total Value across Different Strategies and Metrics (2019-2024)",
    xlabel="Strategies and Metrics",
    ylabel="Portfolio Total Value",
    width=800,
    height=400,
    rot=45  # Rotating the x-axis labels for better readability
).opts(yformatter='%.0f', show_grid=True).opts(ylim=(400000, 750000), fontsize = {"title": 14, "labels": 14, "xticks": 10, "yticks": 10})

In [25]:

# import hvplot.pandas  # Make sure hvplot is available

# # Plotting all 'portfolio_total' columns
# plot = all_portfolio_totals_final_day.hvplot.bar(
#     title="Portfolio Total Value across Different Strategies and Metrics",
#     xlabel="Time",  # Assuming the index of your DataFrames represents time
#     ylabel="Portfolio Total Value",
#     width=800,
#     height=400
# ).opts(yformatter='%.0f', show_grid=True)


In [26]:
plot

:Bars   [index]   (2024-04-01 00:00:00)